В данной статье мы разберём простейшие алгоритмы рекомендательных систем.

Рекомендательные системы используются повсеместно:
1. Онлайн-кинотеатры используют их, чтобы предлагать пользователям новые фильмы.
2. Социальные сети предлагают новых друзей или формируют ленту на основе ваших предпочтений.
3. Музыкальные сервисы, онлайн-радио подбирают музыку специально для вас.
4. Интернет-магазины предлагают товары пользователям, которые могут их заинтересовать.

Рассмотрим построение такой системы на датасете от `GroupLens` - [`MovieLens`](https://grouplens.org/datasets/movielens/):
Это набор данных из `27 000` фильмов и `138 000` пользователей, с общим количеством оценок в 20 миллионов.

Но мы воспользуемся уменьшенной версией для быстроты вычислений: `9 000` фильмов, `700` пользователей, `100 000` оценок.
Скачать напрямую датасет можно по этой [ссылке](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)

In [1]:
# для UNIX систем
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2018-04-08 16:01:15--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918269 (897K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 896.75K   379KB/s    in 2.4s    

2018-04-08 16:01:18 (379 KB/s) - ‘ml-latest-small.zip’ saved [918269/918269]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/movies.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/tags.csv  


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Для начала немного посмотрим на данные
`links.csv` - связь между `id` фильма в датасете и `id` соответствующего фильма на `imdb.com` и `themoviedb.org`;

In [4]:
links = pd.read_csv('./ml-latest-small/links.csv', index_col='movieId')
links.head()

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0


`movies.csv` - описание каждого фильма с его названием и жанрами

In [5]:
movies = pd.read_csv('./ml-latest-small/movies.csv', index_col='movieId')
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


`ratings.csv` - оценки пользователей фильмов с временной отметкой

In [8]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


`tags.csv` - список тегов, которые поставил пользователь фильму, с временной отметкой

In [9]:
tags = pd.read_csv('./ml-latest-small/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


# Формализуем задачу
Имеется множество пользователей и множество фильмов. Для некоторых фильмов конкретный пользователь уже поставил оценку, надо предсказать оценку для остальных фильмов.

Допустим есть таблица с оценками пользоватей:
<center>
<img src="./ratings.png">
<center>
    
А нам нужно как можно точнее предсказать оценки под знаком вопроса:
<center>
<img src="./ratings_predict.png">
<center>

Как из этого получить рекомендации? Очень просто, давайте из всех предсказанных оценок, предложим пользователю с наиболее высоким рейтингом.

Мы будем использовать основную идею коллаборативной фильтрации:
> похожим пользователям обычно нравятся похожие объекты

# Класстеризация пользователей

Самый простой метод:
1. Выберем меру схожести 2 пользователей.
2. Класстеризуем пользователей на группы.
3. Оценка фильма - средняя оценка фильма по группе.